In [ ]:
import os
import pandas as pd
import os
from PIL import Image
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split

import numpy as np
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F
!pip install datasets
from datasets import load_dataset



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os


In [ ]:
! unzip "/content/drive/MyDrive/Biometrics Final Version/FaceShape Dataset.zip"

In [ ]:
train_ds = load_dataset("imagefolder", data_dir='/content/FaceShape Dataset/training_set', split="train")
test_ds = load_dataset("imagefolder", data_dir='/content/FaceShape Dataset/testing_set', split="train")
# label2idx and idx2label

id2label = {id:label for id, label in enumerate(train_ds.features['label'].names)}
label2id = {label:id for id,label in id2label.items()}
# split train, val
splits = test_ds.train_test_split(test_size=0.5, shuffle=True, seed=42)
test_ds, val_ds = splits["train"], splits["test"]
print("Features", train_ds.features)
print("Train", train_ds)
print("Validation", val_ds)
print("Test", test_ds)
print("Num labels", len(label2id))
print("Label2Idx", label2id)
print("Label2Idx", id2label)


In [ ]:
import cv2
import matplotlib.pyplot as plt  
from random import randint
list_idx = [randint(0, len(train_ds)) for i in range(9)]
def display_examples():
    fig = plt.figure(figsize=(12,12))
    fig.suptitle("Some examples of images of the dataset", fontsize=30)
    for i, idx in enumerate(list_idx):
        plt.subplot(3,3,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(train_ds[idx]["image"], cmap=plt.cm.binary)
        plt.xlabel(id2label[train_ds[idx]["label"]])
    plt.show()

display_examples()

In [ ]:
!pip install transformers
from transformers import AutoFeatureExtractor
import torchvision.transforms.functional

from torchvision.transforms import (
    CenterCrop, 
    Compose, 
    Normalize, 
    RandomVerticalFlip,
    RandomResizedCrop, 
    Resize, 
    RandomAdjustSharpness,
    ToTensor,
    ConvertImageDtype,
    RandomPerspective,
    RandomRotation,
    
)

feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/swin-tiny-patch4-window7-224")

normalize = Normalize(mean=0, std=1)
_train_transforms = Compose(
        [

            ToTensor(),
            Resize(list(feature_extractor.size.values())),
            RandomVerticalFlip(p=0.05),
            RandomPerspective(0.05,0.05),
            RandomRotation(5),
            # RandomAdjustSharpness(2, 0.8),
            normalize,
            ConvertImageDtype(torch.float)
        ]
    )

_val_transforms = Compose(
        [
            ToTensor(),
            Resize(list(feature_extractor.size.values())),
            normalize,
            ConvertImageDtype(torch.float)
        ]
    )


import sys
sys.path.append('/content/drive/MyDrive/Biometrics Final Version/')
from landmark_detection_new import Landmarks
landmarks=Landmarks("/content/drive/MyDrive/Biometrics Final Version/shape_predictor_68_face_landmarks.dat")

def train_transforms(examples):
    examples['pixel_values'] = [landmarks.image_mask_concat(np.asarray(image.convert("RGB")),thickness=2)/255 for image in examples['image']]    
    examples['pixel_values'] = [_train_transforms(image) for image in examples['pixel_values']]
    return examples

def val_transforms(examples):
    examples['pixel_values'] = [landmarks.image_mask_concat(np.asarray(image.convert("RGB")),thickness=2)/255 for image in examples['image']]    
    examples['pixel_values'] = [_val_transforms(image) for image in examples['pixel_values']]
    return examples

# Set the transforms
train_ds.set_transform(train_transforms)
val_ds.set_transform(val_transforms)
test_ds.set_transform(val_transforms)

In [ ]:
from torch.utils.data import DataLoader
import torch

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}


In [ ]:
from transformers import SwinForImageClassification, SwinConfig

config = SwinConfig.from_pretrained(
        "microsoft/swin-tiny-patch4-window7-224",
        num_labels=len(label2id),
        label2id=label2id,
        id2label=id2label,
        finetuning_task="image-classification",
        num_channels = 4
    )

model = SwinForImageClassification.from_pretrained(
    "microsoft/swin-tiny-patch4-window7-224",
    config=config,
    ignore_mismatched_sizes=True
)
# model.load_state_dict(torch.load("/content/drive/MyDrive/Biometrics Project/transformers_weights_only.pt")) #uncomment if you want to load pretrained model

In [ ]:
from transformers import TrainingArguments, Trainer
metric_name = "accuracy"
args = TrainingArguments(
    f"faceshape",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    learning_rate=5e-6,
    per_device_train_batch_size=80,
    per_device_eval_batch_size=64,
    num_train_epochs=15,
    weight_decay=0.1,
    load_best_model_at_end=False,
    save_total_limit=1,
    metric_for_best_model=metric_name,
    logging_dir='logs',
    logging_steps=1,
    remove_unused_columns=False,
)

In [ ]:
from datasets import load_metric
import numpy as np

metric = load_metric(metric_name)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
model=torch.load("/content/drive/MyDrive/Biometrics Final Version/transformers_final.pt")
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=feature_extractor,
)

In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

In [ ]:
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "offline"
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
trainer.train()

In [ ]:
PATH = "/content/drive/MyDrive/Biometrics Final Version/transformers_final.pt"

# Save
# torch.save(model, PATH) #uncomment if you want to save the model


In [ ]:
trainer.evaluate()

In [ ]:

outputs = trainer.predict(test_ds)
print(outputs.metrics)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_true = outputs.label_ids
y_pred = outputs.predictions.argmax(1)
labels = train_ds.features['label'].names

In [ ]:

# Classification report
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, target_names=labels))

In [ ]:

# Confusion matrix

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot()

In [ ]:
import cv2
import matplotlib.pyplot as plt
from random import randint

list_idx = [randint(0, len(test_ds)) for i in range(9)]

def display_examples():
    fig = plt.figure(figsize=(12,12))
    fig.suptitle("Some examples of images of the test set", fontsize=30)
    for i,idx in enumerate(list_idx):
        plt.subplot(3,3,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(test_ds[idx]["image"], cmap=plt.cm.binary)
        plt.xlabel("Label: "+id2label[y_true[idx]]+"\nPred: "+id2label[y_pred[idx]])
    plt.show()

display_examples()